In [1]:
import pandas as pd
import numpy as np
from itertools import product
from ast import literal_eval
import seaborn as sns
import random
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import geopandas as gp
import matplotlib.pyplot as plt
import pandas as pd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import sys

In [11]:
n=[0.61,0.58,0.54,0.52,0.50,0.48,0.47,0.45,0.46,0.44]
b=[(x-n[ind+1])/(n[ind+1]) for ind,x in enumerate(n[:-1])]


In [12]:
np.average(b)

0.03726254137135576

In [2]:
pubs_path="../../data/nf_folder/Data/DerivedData/CleanedRawData/pubs_author_funder.csv"
cntry_info_path="../../data/nf_folder/Data/AdditionalData/country_metadata.csv"
flag_path='../../data/nf_folder/Data/AdditionalData/country_WoS_ECI_WB_flags.tsv'

In [3]:
pubs_df=pd.read_csv(pubs_path)
pubs_df['author_country'] = pubs_df['author_country'].apply(literal_eval)
pubs_df['funder_country'] = pubs_df['funder_country'].apply(
    lambda x:literal_eval(x) if pd.notnull(x) else "Not-Funded")
pubs_df.head()

/home/miaoli/conda/envs/funding/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,author_country,funder_country,discipline,year,rel_citation,citation
0,30507292,[Turkey],[Turkey],77,2009,0.485,6
1,30574690,"[United States, France]","[United States, United States, United States, ...",83,2009,0.703,9
2,30574691,[Russia],[Russia],83,2009,0.312,4
3,30585095,"[Spain, Spain]","[Spain, Spain, Spain, Spain, Spain, Spain]",89,2009,0.137,1
4,30585097,"[Germany, Spain]",[Spain],89,2009,0.410,3


In [6]:
pubs_df=pubs_df[pubs_df.year.between(2013,2017)]
pubs_df.head(2)

,id,author_country,funder_country,discipline,year,rel_citation,citation
2212289,48998298,[Australia],"[Australia, Australia]",89,2013,5.02,20
2212290,48998300,"[United Kingdom, Slovakia, Slovakia, United Ki...","[United Kingdom, United Kingdom, United Kingdo...",89,2013,0.00,0


In [23]:
code2name_path='..//..//data//nf_folder//Data//RawData//discipline_code.csv'
disname=pd.read_csv(code2name_path)
disname.columns=['discipline','disname']
disname.head(2)

,discipline,disname
0,1,Agricult & Food Science
1,2,Botany


In [26]:
pubs_df=pubs_df.merge(disname,on='discipline')

In [28]:
pubs_df.head(2)

,id,author_country,funder_country,discipline,year,rel_citation,citation,author_distinct,disname
0,48998298,[Australia],"[Australia, Australia]",89,2013,5.02,20,[Australia],General Mathematics
1,48998300,"[United Kingdom, Slovakia, Slovakia, United Ki...","[United Kingdom, United Kingdom, United Kingdo...",89,2013,0.00,0,"[Slovakia, United Kingdom]",General Mathematics


In [29]:
dscp_cit=pubs_df.groupby(['disname'])['citation'].mean().reset_index()
dscp_cit.head()

,disname,citation
0,Acoustics,6.867428
1,Addictive Diseases,8.351598
2,Aerospace Technology,4.562187
3,Agricult & Food Science,7.211676
4,Allergy,16.330828


In [10]:
pubs_df['author_distinct']=pubs_df['author_country'].apply(lambda x:list(set(x)))
pubs_df.head()

,id,author_country,funder_country,discipline,year,rel_citation,citation,author_distinct
2212289,48998298,[Australia],"[Australia, Australia]",89,2013,5.020,20,[Australia]
2212290,48998300,"[United Kingdom, Slovakia, Slovakia, United Ki...","[United Kingdom, United Kingdom, United Kingdo...",89,2013,0.000,0,"[Slovakia, United Kingdom]"
2212291,48998303,[Croatia],[Croatia],89,2013,0.000,0,[Croatia]
2212292,48998305,[Slovenia],"[Slovenia, Slovenia, Slovenia, Slovenia]",89,2013,0.000,0,[Slovenia]
2212293,48998307,"[South Korea, Croatia]",[South Korea],89,2013,3.765,15,"[South Korea, Croatia]"


In [30]:
cntry_df=pubs_df[['author_distinct','disname','citation']].explode('author_distinct')
cntry_df.head()

,author_distinct,disname,citation
0,Australia,General Mathematics,20
1,Slovakia,General Mathematics,0
1,United Kingdom,General Mathematics,0
2,Croatia,General Mathematics,0
3,Slovenia,General Mathematics,0


In [31]:
cntry_cit=cntry_df.groupby(['author_distinct','disname'])['citation'].mean().reset_index()
cntry_cit.head()

,author_distinct,disname,citation
0,Afghanistan,Addictive Diseases,2.500000
1,Afghanistan,Agricult & Food Science,6.833333
2,Afghanistan,Anesthesiology,4.000000
3,Afghanistan,Applied Chemistry,3.000000
4,Afghanistan,Biochemistry & Molecular Biology,5.000000


In [58]:
test = cntry_cit[cntry_cit.author_distinct=='China']
test=test.merge(dscp_cit, on='disname')
test['d']=test['citation_x']-test['citation_y']

In [59]:
test[test.d>0]

,author_distinct,disname,citation_x,citation_y,d
0,China,Acoustics,8.460626,6.867428,1.593198
2,China,Aerospace Technology,5.067692,4.562187,0.505506
3,China,Agricult & Food Science,8.361935,7.211676,1.150259
4,China,Allergy,18.892473,16.330828,2.561645
5,China,Analytical Chemistry,9.819141,9.658394,0.160747
6,China,Anatomy & Morphology,4.572770,4.468452,0.104318
7,China,Anesthesiology,8.358192,7.845733,0.512459
8,China,Applied Chemistry,7.838228,6.029260,1.808968
10,China,Applied Physics,6.960131,6.794109,0.166022
14,China,Biomedical Engineering,15.009554,12.144605,2.864949
